In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn import model_selection
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


In [2]:
train_df=pd.read_csv('/kaggle/input/30-days-of-ml/train.csv')
test_df=pd.read_csv('/kaggle/input/30-days-of-ml/test.csv')
submission_df=pd.read_csv('/kaggle/input/30-days-of-ml/sample_submission.csv')

In [3]:
train_df.head()


,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [4]:
test_df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,15,B,A,A,A,B,B,A,E,D,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,16,B,B,A,C,B,D,A,E,A,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,17,B,B,A,C,B,C,A,E,C,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [5]:

X_train = train_df.drop(['target'], axis = 1)
y_train = train_df.target

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = test_df.copy()


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
from sklearn.preprocessing import OneHotEncoder

s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()
label_X_test = test_df.copy()
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)
OH_test = OH_X_test.drop(['cat9'], axis = 1)


In [7]:
for col in OH_X_train.columns:
    print(col)

id
cont0
cont1
cont2
cont3
cont4
cont5
cont6
cont7
cont8
cont9
cont10
cont11
cont12
cont13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [8]:
for col in OH_test.columns:
    print(col)

id
cont0
cont1
cont2
cont3
cont4
cont5
cont6
cont7
cont8
cont9
cont10
cont11
cont12
cont13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


A simple XGBoost model

In [9]:
'''
xgb_params = {
    'booster': 'gbtree',
    'n_estimators': 10000,
    'learning_rate': 0.055,
    'reg_lambda': 10,
    'reg_alpha': 26,
    'subsample': 0.9,
    'colsample_bytree': 0.12,
    'max_depth': 3,
}

model=XGBRegressor(**xgb_params)


# Bundle preprocessing for numerical and categorical data

model.fit(OH_X_train, y_train,
             verbose = False,
             eval_set = [(OH_X_valid, y_valid)],
             eval_metric = "rmse",
             early_stopping_rounds = 50)
    
# Preprocessing of validation data, get predictions
preds = model.predict(OH_X_valid)

# Evaluate the model
score = mean_squared_error(preds, y_valid,squared=False)
print(score)
'''

'\nxgb_params = {\n    \'booster\': \'gbtree\',\n    \'n_estimators\': 10000,\n    \'learning_rate\': 0.055,\n    \'reg_lambda\': 10,\n    \'reg_alpha\': 26,\n    \'subsample\': 0.9,\n    \'colsample_bytree\': 0.12,\n    \'max_depth\': 3,\n}\n\nmodel=XGBRegressor(**xgb_params)\n\n\n# Bundle preprocessing for numerical and categorical data\n\nmodel.fit(OH_X_train, y_train,\n             verbose = False,\n             eval_set = [(OH_X_valid, y_valid)],\n             eval_metric = "rmse",\n             early_stopping_rounds = 50)\n    \n# Preprocessing of validation data, get predictions\npreds = model.predict(OH_X_valid)\n\n# Evaluate the model\nscore = mean_squared_error(preds, y_valid,squared=False)\nprint(score)\n'

In [10]:
#OPTUNA
import optuna
def tune(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 5)
    n_estimators = trial.suggest_int("n_estimators", 1, 2000)
    
    xgb_params = {
        'booster': 'gbtree',
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'reg_lambda': reg_lambda,
        'reg_alpha': reg_alpha,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'max_depth': max_depth,
    }

    model=XGBRegressor(**xgb_params)


    # Bundle preprocessing for numerical and categorical data

    model.fit(OH_X_train, y_train,
                 verbose = False,
                 eval_set = [(OH_X_valid, y_valid)],
                 eval_metric = "rmse",
                 early_stopping_rounds = 50)

    # Preprocessing of validation data, get predictions
    preds = model.predict(OH_X_valid)

    # Evaluate the model
    score = mean_squared_error(preds, y_valid,squared=False)
    return score

In [11]:
study = optuna.create_study(direction="minimize")
study.optimize(tune, n_trials=180)

[I 2021-08-30 12:44:46,337] A new study created in memory with name: no-name-6fd80750-46d5-4b40-8119-79105ccc2a7a
[I 2021-08-30 12:48:08,590] Trial 0 finished with value: 0.7214909141208707 and parameters: {'learning_rate': 0.08359466832831551, 'reg_lambda': 0.07402002910519527, 'reg_alpha': 0.00014467570595002327, 'subsample': 0.6996227479032721, 'colsample_bytree': 0.3887250575385539, 'max_depth': 5, 'n_estimators': 1537}. Best is trial 0 with value: 0.7214909141208707.
[I 2021-08-30 12:55:09,459] Trial 1 finished with value: 0.7285667888846915 and parameters: {'learning_rate': 0.012977626307741809, 'reg_lambda': 3.5850613844699873e-07, 'reg_alpha': 1.0410499724869052e-07, 'subsample': 0.6723392166927523, 'colsample_bytree': 0.9365476980158512, 'max_depth': 3, 'n_estimators': 1409}. Best is trial 0 with value: 0.7214909141208707.
[I 2021-08-30 12:56:14,263] Trial 2 finished with value: 0.7384873477379826 and parameters: {'learning_rate': 0.015459859052742636, 'reg_lambda': 63.2365859

In [12]:
best_para = study.best_params
print(best_para)

model = XGBRegressor(**best_para)

model.fit(OH_X_train, y_train)

{'learning_rate': 0.12147381998418816, 'reg_lambda': 8.083327463144941e-05, 'reg_alpha': 22.686986950596438, 'subsample': 0.9548457632743411, 'colsample_bytree': 0.16200441938584798, 'max_depth': 3, 'n_estimators': 1545}


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.16200441938584798, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.12147381998418816, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1545, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=22.686986950596438, reg_lambda=8.083327463144941e-05,
             scale_pos_weight=1, subsample=0.9548457632743411,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
test_preds=model.predict(OH_test)

In [14]:
submission_df['target'] = test_preds
submission_df.to_csv('submission.csv', index=False)